<a href="https://colab.research.google.com/github/mostafijurrahmanmamun/Quantum-Machine-Learning-QML-for-binding-affinity-prediction-and-the-QUBO-formulation-/blob/main/QML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: setup

%load_ext google.colab.data_table
%load_ext autoreload
%autoreload 2


The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# %%capture
# #@title 1. Setup: Install Necessary Libraries
# # Run this cell first to install all required packages.
!pip install pennylane scikit-learn numpy rdkit-pypi pyqubo dwave-ocean-sdk simanneal

print("Libraries installed successfully!")

Libraries installed successfully!


In [ ]:

#@title 2. Imports and Basic Configuration
import pennylane as qml
from pennylane import numpy as np # Use PennyLane's wrapped numpy for autograd
import matplotlib.pyplot as plt

# Scikit-learn
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA # For potential feature reduction

# RDKit (for cheminformatics - conceptual use here)
from rdkit import Chem
from rdkit.Chem import AllChem

# QUBO related
from pyqubo import Array, Placeholder, solve_qubo
import neal # D-Wave's simulated annealer
from simanneal import Annealer # For classical SA comparison
import dimod # For BQM

# Other
import time
import pandas as pd

# --- Configuration ---
NUM_QUBITS_QML = 4
NUM_FEATURES_QML = 4 # After PCA or selection, matching NUM_QUBITS_QML for AngleEmbedding
QML_ANSATZ_LAYERS = 2 # Number of layers in the VQC ansatz

NUM_EXCIPIENTS_QUBO = 20 # m in the paper
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

print("Imports successful and basic configuration set.")

Imports successful and basic configuration set.


In [ ]:
#@title 3. Data Generation/Loading (Placeholder for Davis Dataset)

# --- Placeholder Data Generation ---
# In a real scenario, you would:
# 1. Load the Davis dataset (SMILES strings for ligands, protein info, Kd values).
# 2. Calculate molecular descriptors (e.g., Morgan fingerprints using RDKit).
# 3. Apply PCA or feature selection to get NUM_FEATURES_QML (e.g., 4) features.
# 4. Convert Kd to pKd (e.g., -log10(Kd_molar)).

N_SAMPLES = 100 # Small dataset for quick demo

# Placeholder features (e.g., after PCA from molecular descriptors)
X_placeholder = np.random.rand(N_SAMPLES, NUM_FEATURES_QML)

# Placeholder target binding affinities (pKd values)
# Simulating a relationship: y = sum(features) + noise
y_placeholder = np.sum(X_placeholder, axis=1) + 0.1 * np.random.randn(N_SAMPLES)

# Scale features and target (common practice for ML/QML)
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X_placeholder)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y_placeholder.reshape(-1, 1)).flatten()


print(f"Generated placeholder data: X_scaled shape: {X_scaled.shape}, y_scaled shape: {y_scaled.shape}")

# --- Conceptual RDKit + PCA (Illustrative - Not run with placeholder data) ---
def get_morgan_fingerprints_and_pca(smiles_list, n_components=4):
    """
    Conceptual function to get Morgan fingerprints and apply PCA.
    This is for illustration; you'd need a real list of SMILES.
    """
    if not smiles_list:
        print("SMILES list is empty. Cannot generate fingerprints.")
        return None

    mols = [Chem.MolFromSmiles(s) for s in smiles_list]
    mols = [m for m in mols if m is not None] # Filter out invalid SMILES

    if not mols:
        print("No valid molecules from SMILES list. Cannot generate fingerprints.")
        return None

    fps = []
    for mol in mols:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
        arr = np.zeros((1,), dtype=int) # For RDKit to convert to numpy
        Chem.DataStructs.ConvertToNumpyArray(fp, arr)
        fps.append(arr)

    if not fps:
        print("No fingerprints generated.")
        return None

    fps_np = np.array(fps)
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(fps_np)
    print(f"PCA explained variance ratio: {pca.explained_variance_ratio_}")
    return principal_components

# Example SMILES (replace with your actual data)
# example_smiles = ["CCO", "c1ccccc1C(=O)O", "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"]
# if you had real SMILES, you could call:
# X_real_features = get_morgan_fingerprints_and_pca(example_smiles, n_components=NUM_FEATURES_QML)
# For this demo, we continue with X_scaled (placeholder)

Generated placeholder data: X_scaled shape: (100, 4), y_scaled shape: (100,)


In [ ]:
#@title 4. Variational Quantum Circuit (VQC) Model - PennyLane

# Define the quantum device
dev = qml.device("default.qubit", wires=NUM_QUBITS_QML)

# Define the VQC layers
def feature_map(x):
    """Encodes classical features into quantum states."""
    qml.AngleEmbedding(features=x, wires=range(NUM_QUBITS_QML), rotation='X')

def ansatz_layer(weights):
    """A single layer of the variational ansatz."""
    # Example: BasicEntanglerLayers structure
    for i in range(NUM_QUBITS_QML):
        qml.RY(weights[i], wires=i)
    # This loop was causing the IndentationError due to missing code block
    for i in range(NUM_QUBITS_QML - 1):
        qml.CNOT(wires=[i, i + 1])
    if NUM_QUBITS_QML > 1: # For ring connectivity if more than 1 qubit
        qml.CNOT(wires=[NUM_QUBITS_QML - 1, 0])


@qml.qnode(dev, interface="autograd")
def vqc_model(weights, features):
    """The full VQC model."""
    feature_map(features)
    for w_layer in weights: # weights is expected to be (num_layers, num_params_per_layer)
        ansatz_layer(w_layer)

    # Expectation value of Z0 Z1 Z2 Z3 (as per paper's Z^n)
    # For simplicity, let's use expectation of Z on the first qubit as an example output.
    # The paper uses Z^n, which for n=4 is Z0*Z1*Z2*Z3.
    # This requires qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3)
    # For a simpler regression output, often a single qubit observable or sum is used.
    # Let's try to match the paper's Z^n observable.
    if NUM_QUBITS_QML == 4:
      obs = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3)
    else: # Fallback for different qubit numbers
      obs = qml.PauliZ(0)
    return qml.expval(obs)

# Cost function (MSE)
def square_loss(targets, predictions):
    loss = 0
    for t, p in zip(targets, predictions):
        loss += (t - p) ** 2
    loss = loss / len(targets)
    return loss

def cost_function(weights, X_batch, y_batch):
    predictions = [vqc_model(weights, x) for x in X_batch]
    return square_loss(y_batch, predictions)

# Initialize weights for the VQC
# Each layer of our example ansatz_layer has NUM_QUBITS_QML parameters (for RY gates)
params_per_layer = NUM_QUBITS_QML
initial_weights = np.random.randn(QML_ANSATZ_LAYERS, params_per_layer, requires_grad=True)

print(f"VQC model defined. Initial weights shape: {initial_weights.shape}")
# Test the VQC with one sample
# print(f"Test VQC output for one sample: {vqc_model(initial_weights, X_scaled)}")

VQC model defined. Initial weights shape: (2, 4)


In [ ]:
#@title 5. QML Model Training (Illustrative)

# Split data for training and testing
X_train_qml, X_test_qml, y_train_qml, y_test_qml = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=RANDOM_SEED
)

# Optimizer
opt = qml.AdamOptimizer(stepsize=0.1)
batch_size = 10
epochs = 20 # Keep low for quick demo

weights_qml = np.copy(initial_weights) # Start with fresh weights

print("Starting QML model training (illustrative)...")
for epoch in range(epochs):
    for i in range(0, len(X_train_qml), batch_size):
        X_batch = X_train_qml[i : i + batch_size]
        y_batch = y_train_qml[i : i + batch_size]
        weights_qml, _, _ = opt.step(cost_function, weights_qml, X_batch, y_batch) # Unpack cost from opt.step

    # Cost on training data
    train_predictions = [vqc_model(weights_qml, x) for x in X_train_qml]
    current_cost = square_loss(y_train_qml, train_predictions)
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Training Cost: {current_cost:.4f}")

print("QML training finished.")

# Evaluate QML model
qml_predictions_test = np.array([vqc_model(weights_qml, x) for x in X_test_qml])
qml_mse = mean_squared_error(y_test_qml, qml_predictions_test)
qml_r2 = r2_score(y_test_qml, qml_predictions_test)

print(f"\n--- QML Model Evaluation (on scaled test data) ---")
print(f"MSE: {qml_mse:.4f}")
print(f"R2 Score: {qml_r2:.4f}")

# To get predictions in original scale (if y was scaled)
# qml_predictions_original_scale = scaler_y.inverse_transform(qml_predictions_test.reshape(-1,1)).flatten()
# y_test_original_scale = scaler_y.inverse_transform(y_test_qml.reshape(-1,1)).flatten()
# qml_mse_orig = mean_squared_error(y_test_original_scale, qml_predictions_original_scale)
# print(f"MSE (original scale): {qml_mse_orig:.4f}")

Starting QML model training (illustrative)...
Epoch 5/20, Training Cost: 1.0236
Epoch 10/20, Training Cost: 1.0178
Epoch 15/20, Training Cost: 1.0170
Epoch 20/20, Training Cost: 1.0169
QML training finished.

--- QML Model Evaluation (on scaled test data) ---
MSE: 0.7116
R2 Score: -0.0493


In [ ]:
#@title 6. Classical Baselines (Random Forest & SVR)

# Use the same scaled data split as QML for fair comparison
X_train_cl, X_test_cl, y_train_cl, y_test_cl = X_train_qml, X_test_qml, y_train_qml, y_test_qml

# --- Random Forest Regressor ---
print("\n--- Random Forest Regressor ---")
rf_model = RandomForestRegressor(random_state=RANDOM_SEED)
# Conceptual GridSearchCV (can be slow for demo)
# param_grid_rf = {'n_estimators': [1, 2], 'max_depth': [None, 10]}
# grid_rf = GridSearchCV(rf_model, param_grid_rf, cv=3, scoring='neg_mean_squared_error')
# grid_rf.fit(X_train_cl, y_train_cl)
# rf_model = grid_rf.best_estimator_
# print(f"Best RF params: {grid_rf.best_params_}")

rf_model.fit(X_train_cl, y_train_cl) # Fit with default or best params
rf_predictions_test = rf_model.predict(X_test_cl)
rf_mse = mean_squared_error(y_test_cl, rf_predictions_test)
rf_r2 = r2_score(y_test_cl, rf_predictions_test)
print(f"RF Test MSE: {rf_mse:.4f}, R2 Score: {rf_r2:.4f}")


# --- Support Vector Regressor (SVR) ---
print("\n--- Support Vector Regressor ---")
svr_model = SVR()
# Conceptual GridSearchCV
# param_grid_svr = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}
# grid_svr = GridSearchCV(svr_model, param_grid_svr, cv=3, scoring='neg_mean_squared_error')
# grid_svr.fit(X_train_cl, y_train_cl)
# svr_model = grid_svr.best_estimator_
# print(f"Best SVR params: {grid_svr.best_params_}")

svr_model.fit(X_train_cl, y_train_cl) # Fit with default or best params
svr_predictions_test = svr_model.predict(X_test_cl)
svr_mse = mean_squared_error(y_test_cl, svr_predictions_test)
svr_r2 = r2_score(y_test_cl, svr_predictions_test)
print(f"SVR Test MSE: {svr_mse:.4f}, R2 Score: {svr_r2:.4f}")

# --- Summary Table (Conceptual, based on paper's format) ---
# The paper reports MSE +/- std and R2 from 5-fold CV.
# The above is a single train/test split. For full CV:
# kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
# qml_mses, rf_mses, svr_mses =,,
# qml_r2s, rf_r2s, svr_r2s =,,
# for train_idx, test_idx in kf.split(X_scaled):
#     X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
#     y_train, y_test = y_scaled[train_idx], y_scaled[test_idx]
#     #... retrain and evaluate each model...
# This full CV loop is omitted for brevity in this direct script.

print("\nNote: The paper's results (MSE +/- std, R2) are from 5-fold Cross-Validation.")
print("The evaluation above is based on a single train/test split for brevity.")
print("For a robust comparison, implement full k-fold CV for all models.")


--- Random Forest Regressor ---
RF Test MSE: 0.1668, R2 Score: 0.7541

--- Support Vector Regressor ---
SVR Test MSE: 0.0738, R2 Score: 0.8912

Note: The paper's results (MSE +/- std, R2) are from 5-fold Cross-Validation.
The evaluation above is based on a single train/test split for brevity.
For a robust comparison, implement full k-fold CV for all models.


In [ ]:
#@title 7. QUBO Definition (using PyQUBO)

# Placeholder coefficients for the QUBO
# H = -sum(b_i*x_i) + lambda1*sum(t_ij*x_i*x_j) + lambda2*sum(s_i*x_i)
# x_i are binary variables (0 or 1) for NUM_EXCIPIENTS_QUBO

# Create binary variables for each excipient
x = Array.create('x', shape=NUM_EXCIPIENTS_QUBO, vartype='BINARY')

# Placeholder coefficients (randomly generated for this demo)
# In a real scenario, these would come from experimental data, QSPR models, literature.
# IMPORTANT: Ensure these are standard numpy arrays/floats, NOT PennyLane tensors
# using the original numpy import alias (import numpy as np) is safer here.
# Since we imported pennylane.numpy as np, let's convert them explicitly.

# *** FIX START ***
# Temporarily use standard numpy for generating coefficients to avoid PennyLane tensors
import numpy as standard_np

b_coeffs_np = standard_np.random.rand(NUM_EXCIPIENTS_QUBO).astype(float)      # Bioavailability (higher is better, so -b_i in QUBO)
s_coeffs_np = (standard_np.random.rand(NUM_EXCIPIENTS_QUBO) * 0.5).astype(float) # Stability penalty (lower is better)
t_coeffs_np = (standard_np.random.rand(NUM_EXCIPIENTS_QUBO, NUM_EXCIPIENTS_QUBO) * 0.2).astype(float) # Pairwise toxicity (lower is better)

# Convert back to pennylane.numpy if needed elsewhere, but not for QUBO construction
# The code below uses these standard numpy arrays directly, so no conversion back needed immediately.
# *** FIX END ***


standard_np.fill_diagonal(t_coeffs_np, 0) # No self-toxicity in pairwise term
t_coeffs_np = standard_np.triu(t_coeffs_np, k=1) # Ensure t_ij = t_ji and only count pairs once

# Placeholder lambda values (these need careful tuning via grid search in practice)
lambda1_val = Placeholder('lambda1') # For toxicity
lambda2_val = Placeholder('lambda2') # For stability

# Construct the QUBO Hamiltonian using PyQUBO
# Bioavailability term (maximize sum(b_i*x_i) -> minimize -sum(b_i*x_i))
# Use standard numpy arrays/floats here
term_bioavailability = -sum(b_coeffs_np[i] * x[i] for i in range(NUM_EXCIPIENTS_QUBO))

# Toxicity term (minimize sum(t_ij*x_i*x_j))
# Use standard numpy arrays/floats here
term_toxicity = sum(t_coeffs_np[i, j] * x[i] * x[j]
                    for i in range(NUM_EXCIPIENTS_QUBO)
                    for j in range(i + 1, NUM_EXCIPIENTS_QUBO)) # Iterate over upper triangle

# Stability term (minimize sum(s_i*x_i))
# Use standard numpy arrays/floats here
term_stability = sum(s_coeffs_np[i] * x[i] for i in range(NUM_EXCIPIENTS_QUBO))

# Full Hamiltonian
# This combination now involves PyQUBO expressions and Placeholders, which is compatible
H = term_bioavailability + lambda1_val * term_toxicity + lambda2_val * term_stability

# Compile the model to get QUBO coefficients (Q matrix)
# Substitute placeholder lambda values for compilation
feed_dict = {'lambda1': 1.0, 'lambda2': 1.0} # Example lambda values
model = H.compile()
Q, offset = model.to_qubo(feed_dict=feed_dict)

print(f"QUBO defined for {NUM_EXCIPIENTS_QUBO} excipients.")
print(f"Number of variables in Q: {len(Q)}")
# print("Q matrix (sample):", dict(list(Q.items())[:5])) # Show a few terms
# print("Offset:", offset)

QUBO defined for 20 excipients.
Number of variables in Q: 210


In [ ]:
#@title 8. QUBO Solving

# --- 8.1 Using D-Wave's Simulated Annealer (Neal) ---
# This simulates a quantum annealer's behavior classically.
# For actual D-Wave QPU, use: from dwave.system import DWaveSampler, EmbeddingComposite
# and sampler = EmbeddingComposite(DWaveSampler(token='YOUR_DWAVE_API_TOKEN'))

sampler_neal = neal.SimulatedAnnealingSampler()
start_time_neal = time.time()
sampleset_neal = sampler_neal.sample_qubo(Q, num_reads=100) # num_reads for SA means num_runs
end_time_neal = time.time()
neal_time = end_time_neal - start_time_neal

best_solution_neal = sampleset_neal.first.sample
best_energy_neal = sampleset_neal.first.energy

print("\n--- D-Wave Neal (Simulated Annealer) ---")
print(f"Time taken: {neal_time:.4f} seconds")
print(f"Best energy: {best_energy_neal:.4f}")
# print(f"Best solution (x_i values): {best_solution_neal}")
num_selected_neal = sum(best_solution_neal[f'x[{i}]'] for i in range(NUM_EXCIPIENTS_QUBO))
print(f"Number of excipients selected by Neal: {num_selected_neal}")


# --- 8.2 Classical Simulated Annealing (using simanneal library) ---
# This is a separate classical SA implementation for comparison.
class QUBOAnnealer(Annealer):
    def __init__(self, state, qubo_dict):
        self.qubo_dict = qubo_dict
        super(QUBOAnnealer, self).__init__(state)

    def move(self):
        # Flip a random bit
        idx_to_flip = np.random.randint(0, len(self.state))
        self.state[idx_to_flip] = 1 - self.state[idx_to_flip]

    def energy(self):
        # Calculate QUBO energy: x^T Q x
        current_energy = 0.0
        # Linear terms (diagonal of Q)
        for i in range(len(self.state)):
            if (f'x[{i}]', f'x[{i}]') in self.qubo_dict:
                 current_energy += self.qubo_dict[(f'x[{i}]', f'x[{i}]')] * self.state[i]
        # Quadratic terms (off-diagonal of Q)
        for i in range(len(self.state)):
            for j in range(i + 1, len(self.state)):
                if (f'x[{i}]', f'x[{j}]') in self.qubo_dict:
                    current_energy += self.qubo_dict[(f'x[{i}]', f'x[{j}]')] * self.state[i] * self.state[j]
        return current_energy + offset # Add the offset from PyQUBO compilation

# Initial state for classical SA (random binary string)
initial_state_sa = np.random.randint(0, 2, size=NUM_EXCIPIENTS_QUBO).tolist()

# Convert Q from PyQUBO format (dict of tuples) to what QUBOAnnealer expects
# Q from model.to_qubo() is already in a suitable dict format: {('x[i]', 'x[j]'): coeff}
# We need to ensure the keys match the variable naming if we directly use `model.variables`
# For simplicity, let's use the Q dict directly from model.to_qubo()

sa = QUBOAnnealer(initial_state_sa, Q)
sa.set_schedule(sa.auto(minutes=0.01)) # Short run for demo; tune Tmax, Tmin, steps for real use

start_time_sa = time.time()
best_state_sa, best_energy_sa = sa.anneal()
end_time_sa = time.time()
sa_time = end_time_sa - start_time_sa

print("\n--- Classical Simulated Annealer (simanneal library) ---")
print(f"Time taken: {sa_time:.4f} seconds")
print(f"Best energy: {best_energy_sa:.4f}")
# print(f"Best solution (x_i values): {best_state_sa}")
num_selected_sa = sum(best_state_sa)
print(f"Number of excipients selected by classical SA: {num_selected_sa}")

# --- Benchmarking "5x faster" (Conceptual) ---
if sa_time > 0 and neal_time > 0: # Avoid division by zero
    speedup_factor = sa_time / neal_time
    print(f"\nConceptual Speedup (Neal vs. Classical SA): {speedup_factor:.2f}x")
    print("(Note: This is illustrative. Real speedup depends on hardware, problem, and tuning.)")
else:
    print("\nCould not calculate speedup factor due to zero time for one of the solvers (likely too fast for this small problem).")

print("\nReminder: The paper's '5x faster' claim is for D-Wave Leap (actual QPU) vs. a specific classical SA.")
print("Neal is a classical simulator. For QPU, use DWaveSampler with API token.")

 Temperature        Energy    Accept   Improve     Elapsed   Remaining



--- D-Wave Neal (Simulated Annealer) ---
Time taken: 0.0730 seconds
Best energy: -2.5210
Number of excipients selected by Neal: 6


 Temperature        Energy    Accept   Improve     Elapsed   Remaining



--- Classical Simulated Annealer (simanneal library) ---
Time taken: 1.2056 seconds
Best energy: -6.6075
Number of excipients selected by classical SA: 15

Conceptual Speedup (Neal vs. Classical SA): 16.52x
(Note: This is illustrative. Real speedup depends on hardware, problem, and tuning.)

Reminder: The paper's '5x faster' claim is for D-Wave Leap (actual QPU) vs. a specific classical SA.
Neal is a classical simulator. For QPU, use DWaveSampler with API token.


In [ ]:
#@title 9. Pareto Frontier and Lambda Grid Search (Conceptual)

print("\n--- Pareto Frontier and Lambda Grid Search (Conceptual) ---")
# To find the Pareto frontier, you would iterate over a grid of lambda1 and lambda2 values.
# For each (lambda1, lambda2) pair:
# 1. Re-compile the PyQUBO model: Q_new, offset_new = model.to_qubo(feed_dict={'lambda1': l1, 'lambda2': l2})
# 2. Solve the new QUBO (e.g., using sampler_neal.sample_qubo(Q_new,...))
# 3. Decode the solution and calculate the individual objective values:
#    - Total Bioavailability = sum(b_coeffs[i] * x_sol[i])
#    - Total Toxicity = sum(t_coeffs[i,j] * x_sol[i] * x_sol[j])
#    - Total Stability = sum(s_coeffs[i] * x_sol[i])
# 4. Collect all unique solutions and their objective values.
# 5. Apply a non-dominated sorting algorithm to find the Pareto-optimal set.

# Example lambda ranges (illustrative)
lambda1_range = [0.1, 0.5, 1.0, 2.0, 5.0]
#@title 9. Pareto Frontier and Lambda Grid Search (Conceptual)

print("\n--- Pareto Frontier and Lambda Grid Search (Conceptual) ---")
# To find the Pareto frontier, you would iterate over a grid of lambda1 and lambda2 values.
# For each (lambda1, lambda2) pair:
# 1. Re-compile the PyQUBO model: Q_new, offset_new = model.to_qubo(feed_dict={'lambda1': l1, 'lambda2': l2})
# 2. Solve the new QUBO (e.g., using sampler_neal.sample_qubo(Q_new,...))
# 3. Decode the solution and calculate the individual objective values:
#    - Total Bioavailability = sum(b_coeffs[i] * x_sol[i])
#    - Total Toxicity = sum(t_coeffs[i,j] * x_sol[i] * x_sol[j])
#    - Total Stability = sum(s_coeffs[i] * x_sol[i])
# 4. Collect all unique solutions and their objective values.
# 5. Apply a non-dominated sorting algorithm to find the Pareto-optimal set.

# Example lambda ranges (illustrative)
lambda1_range = [0.1, 0.5, 1.0, 2.0, 5.0]
lambda2_range = [0.1, 0.5, 1.0, 2.0, 5.0]

# Initialize an empty list to store results
all_solutions_objectives = [] # Store (bioavailability, toxicity, stability, solution_vector)

# Conceptual loop (not run for brevity)
# for l1_val_current in lambda1_range:
#     for l2_val_current in lambda2_range:
#         current_feed_dict = {'lambda1': l1_val_current, 'lambda2': l2_val_current}
#         Q_curr, offset_curr = model.to_qubo(feed_dict=current_feed_dict)
#         sampleset_curr = sampler_neal.sample_qubo(Q_curr, num_reads=50)
#         sol_dict = sampleset_curr.first.sample
#         sol_vector = [sol_dict[f'x[{i}]'] for i in range(NUM_EXCIPIENTS_QUBO)]
#
#         # Calculate individual objectives for this solution
#         current_bio = sum(b_coeffs[i] * sol_vector[i] for i in range(NUM_EXCIPIENTS_QUBO))
#         current_tox = sum(t_coeffs[i, j] * sol_vector[i] * sol_vector[j]
#                             for i in range(NUM_EXCIPIENTS_QUBO)
#                             for j in range(i + 1, NUM_EXCIPIENTS_QUBO))
#         current_stab = sum(s_coeffs[i] * sol_vector[i] for i in range(NUM_EXCIPIENTS_QUBO))
#
#         all_solutions_objectives.append({
#             "bioavailability": current_bio, # Maximize this
#             "toxicity": current_tox,       # Minimize this
#             "stability": current_stab,     # Minimize this
#             "solution_vector": sol_vector,
#             "lambdas": (l1_val_current, l2_val_current)
#         })
#
# # After collecting all_solutions_objectives, apply non-dominated sorting.
# # This part is non-trivial to implement fully here.
# print(f"Conceptual: Collected {len(all_solutions_objectives)} solutions from lambda grid search (if run).")
print("A full Pareto frontier generation involves solving the QUBO for many lambda combinations")
print("and then filtering for non-dominated solutions.")
# 1. Re-compile the PyQUBO model: Q_new, offset_new = model.to_qubo(feed_dict={'lambda1': l1, 'lambda2': l2})
# 2. Solve the new QUBO (e.g., using sampler_neal.sample_qubo(Q_new,...))
# 3. Decode the solution and calculate the individual objective values:
#    - Total Bioavailability = sum(b_coeffs[i] * x_sol[i])
#    - Total Toxicity = sum(t_coeffs[i,j] * x_sol[i] * x_sol[j])
#    - Total Stability = sum(s_coeffs[i] * x_sol[i])
# 4. Collect all unique solutions and their objective values.
# 5. Apply a non-dominated sorting algorithm to find the Pareto-optimal set.

# Example lambda ranges (illustrative)
lambda1_range = [0.1, 0.5, 1.0, 2.0, 5.0]
lambda2_range = [0.1, 0.5, 1.0, 2.0, 5.0]

# Initialize an empty list to store results
all_solutions_objectives = [] # Store (bioavailability, toxicity, stability, solution_vector)

# Conceptual loop (not run for brevity)
# for l1_val_current in lambda1_range:
#     for l2_val_current in lambda2_range:
#         current_feed_dict = {'lambda1': l1_val_current, 'lambda2': l2_val_current}
#         Q_curr, offset_curr = model.to_qubo(feed_dict=current_feed_dict)
#         sampleset_curr = sampler_neal.sample_qubo(Q_curr, num_reads=50)
#         sol_dict = sampleset_curr.first.sample
#         sol_vector = [sol_dict[f'x[{i}]'] for i in range(NUM_EXCIPIENTS_QUBO)]
#
#         # Calculate individual objectives for this solution
#         current_bio = sum(b_coeffs[i] * sol_vector[i] for i in range(NUM_EXCIPIENTS_QUBO))
#         current_tox = sum(t_coeffs[i, j] * sol_vector[i] * sol_vector[j]
#                             for i in range(NUM_EXCIPIENTS_QUBO)
#                             for j in range(i + 1, NUM_EXCIPIENTS_QUBO))
#         current_stab = sum(s_coeffs[i] * sol_vector[i] for i in range(NUM_EXCIPIENTS_QUBO))
#
#         all_solutions_objectives.append({
#             "bioavailability": current_bio, # Maximize this
#             "toxicity": current_tox,       # Minimize this
#             "stability": current_stab,     # Minimize this
#             "solution_vector": sol_vector,
#             "lambdas": (l1_val_current, l2_val_current)
#         })
#
# # After collecting all_solutions_objectives, apply non-dominated sorting.
# # This part is non-trivial to implement fully here.
# print(f"Conceptual: Collected {len(all_solutions_objectives)} solutions from lambda grid search (if run).")
print("A full Pareto frontier generation involves solving the QUBO for many lambda combinations")
print("and then filtering for non-dominated solutions.")
# for l1_val_current in lambda1_range:
#     for l2_val_current in lambda2_range:
#         current_feed_dict = {'lambda1': l1_val_current, 'lambda2': l2_val_current}
#         Q_curr, offset_curr = model.to_qubo(feed_dict=current_feed_dict)
#         sampleset_curr = sampler_neal.sample_qubo(Q_curr, num_reads=50)
#         sol_dict = sampleset_curr.first.sample
#         sol_vector = [sol_dict[f'x[{i}]'] for i in range(NUM_EXCIPIENTS_QUBO)]
#
#         # Calculate individual objectives for this solution
#         current_bio = sum(b_coeffs[i] * sol_vector[i] for i in range(NUM_EXCIPIENTS_QUBO))
#         current_tox = sum(t_coeffs[i, j] * sol_vector[i] * sol_vector[j]
#                             for i in range(NUM_EXCIPIENTS_QUBO)
#                             for j in range(i + 1, NUM_EXCIPIENTS_QUBO))
#         current_stab = sum(s_coeffs[i] * sol_vector[i] for i in range(NUM_EXCIPIENTS_QUBO))
#
#         all_solutions_objectives.append({
#             "bioavailability": current_bio, # Maximize this
#             "toxicity": current_tox,       # Minimize this
#             "stability": current_stab,     # Minimize this
#             "solution_vector": sol_vector,
#             "lambdas": (l1_val_current, l2_val_current)
#         })
#
# # After collecting all_solutions_objectives, apply non-dominated sorting.
# # This part is non-trivial to implement fully here.
# print(f"Conceptual: Collected {len(all_solutions_objectives)} solutions from lambda grid search (if run).")
print("A full Pareto frontier generation involves solving the QUBO for many lambda combinations")
print("and then filtering for non-dominated solutions.")


--- Pareto Frontier and Lambda Grid Search (Conceptual) ---

--- Pareto Frontier and Lambda Grid Search (Conceptual) ---
A full Pareto frontier generation involves solving the QUBO for many lambda combinations
and then filtering for non-dominated solutions.
A full Pareto frontier generation involves solving the QUBO for many lambda combinations
and then filtering for non-dominated solutions.
A full Pareto frontier generation involves solving the QUBO for many lambda combinations
and then filtering for non-dominated solutions.


In [ ]:
#@title 10. Conclusion and Next Steps

print("\n--- Conclusion and Next Steps ---")
print("This Colab notebook provides a conceptual framework for the experiments in the paper.")

print("\nFor QML Binding Affinity Prediction:")
print("- We used placeholder data. Real implementation requires Davis dataset processing, molecular descriptor calculation (e.g., RDKit), and PCA.")
print("- VQC training was illustrative. Robust results need full k-fold CV and potentially more epochs/tuning.")
print("- Classical baselines (RF, SVR) should also be tuned with GridSearchCV and evaluated with k-fold CV.")

print("\nFor QUBO Drug Formulation:")
print("- QUBO coefficients (b_i, t_ij, s_i) were placeholders. These need to be derived from domain knowledge or predictive models.")
print("- D-Wave Neal was used for simulated annealing. For actual quantum annealing, use DWaveSampler with an API token.")
print("- The classical SA comparison and '5x faster' claim require careful, matched benchmarking against a well-tuned classical SA and specific D-Wave hardware.")
print("- Pareto frontier generation requires a systematic grid search over lambda parameters and non-dominated sorting.")

print("\nTo extend this work:")
print("1. Integrate real data for QML.")
print("2. Implement robust k-fold CV and statistical comparisons for QML.")
print("3. Develop realistic QUBO coefficients.")
print("4. If D-Wave access is available, run QUBOs on a QPU.")
print("5. Implement the full Pareto frontier generation and analysis.")
print("6. Rigorously benchmark QA vs. classical SA for the QUBOs.")


--- Conclusion and Next Steps ---
This Colab notebook provides a conceptual framework for the experiments in the paper.

For QML Binding Affinity Prediction:
- We used placeholder data. Real implementation requires Davis dataset processing, molecular descriptor calculation (e.g., RDKit), and PCA.
- VQC training was illustrative. Robust results need full k-fold CV and potentially more epochs/tuning.
- Classical baselines (RF, SVR) should also be tuned with GridSearchCV and evaluated with k-fold CV.

For QUBO Drug Formulation:
- QUBO coefficients (b_i, t_ij, s_i) were placeholders. These need to be derived from domain knowledge or predictive models.
- D-Wave Neal was used for simulated annealing. For actual quantum annealing, use DWaveSampler with an API token.
- The classical SA comparison and '5x faster' claim require careful, matched benchmarking against a well-tuned classical SA and specific D-Wave hardware.
- Pareto frontier generation requires a systematic grid search over lambda 